## Imports

In [2]:
import pandas as pd
from event_recurrance import *
from sample_event_list import existing_events
from test_cases import test_cases

## 1. Run Test

In [3]:
n = 1
test_results_df = pd.DataFrame()
test_tittles_dic = {}

# Test Results.
for tittle, new_event_data in test_cases.items():

    # Summary
    print(f"Test Case {n}: {tittle}")
    scenario = count_weekly_events(new_event_data, existing_events)
    print(summary_weekly_events(scenario),"\n")

    # Add test data to dataframe
    temp_df = pd.DataFrame(list(scenario.items()), columns=["Week Start Date", "Event Count"])
    temp_df["Test Number"] = n
    test_results_df = pd.concat([test_results_df, temp_df], ignore_index=True)

    # Test tittle data
    #test_tittles_dic[n] = tittle

    n += 1

test_results_df.head(5)

Test Case 1: Adding a new weekly event on Mondays
Total events: 977 | Total weeks: 42 | Average events per week:  23 

Test Case 2: Adding a new bi-weekly event on Wednesdays
Total events: 977 | Total weeks: 42 | Average events per week:  23 

Test Case 3: Adding a new monthly event on the 15th
Total events: 987 | Total weeks: 42 | Average events per week:  24 

Test Case 4: Adding a weekly event overlapping multiple existing events
Total events: 977 | Total weeks: 42 | Average events per week:  23 

Test Case 5: Adding an event far into the future
Total events: 994 | Total weeks: 42 | Average events per week:  24 

Test Case 6: Weekly Salary Deposit
Total events: 957 | Total weeks: 42 | Average events per week:  23 

Test Case 7: Bi-weekly Grocery Shopping
Total events: 957 | Total weeks: 42 | Average events per week:  23 

Test Case 8: Monthly Rent Payment
Total events: 957 | Total weeks: 42 | Average events per week:  23 

Test Case 9: Electric Bill Payment
Total events: 987 | Total

,Week Start Date,Event Count,Test Number
0,2024-03-04,15,1
1,2024-03-11,15,1
2,2024-03-18,24,1
3,2024-03-25,23,1
4,2024-04-01,23,1


## 2. Test Cases Data

In [8]:
# Convert to DataFrame
test_cases_data_df = pd.DataFrame.from_dict(test_cases, orient='index')

# Reset index to include case names and add case numbers
test_cases_data_df.reset_index(inplace=True)
test_cases_data_df.rename(columns={'index': 'case_description'}, inplace=True)
test_cases_data_df.insert(0, 'case_number', range(1, len(test_cases_data_df) + 1))

test_cases_data_df.head()

,case_number,case_description,name,start_date,recurrent_type,interval,days_of_week,days_of_month,end_date
0,1,Adding a new weekly event on Mondays,Monday Sync,2024-03-04,n-weekly,1.0,[0],NaN,NaT
1,2,Adding a new bi-weekly event on Wednesdays,Bi-Weekly Stand-up,2024-03-06,n-weekly,2.0,[2],NaN,NaT
2,3,Adding a new monthly event on the 15th,Mid-Month Review,2024-03-15,monthly,NaN,NaN,[15],NaT
3,4,Adding a weekly event overlapping multiple exi...,Busy Monday,2024-03-04,n-weekly,1.0,"[0, 3]",NaN,NaT
4,5,Adding an event far into the future,Yearly Review,2025-03-01,monthly,NaN,NaN,[1],NaT


## 3. Existing events data

In [13]:
# Convert the list of events into a DataFrame
existing_events_df = pd.DataFrame([
    {
        "Name": event.name,
        "Start Date": event.start_date.strftime("%Y-%m-%d"),
        "End Date": event.end_date.strftime("%Y-%m-%d") if event.end_date else None,
        "Recurrent Type": event.recurrent_type,
        "Days of Month": event.days_of_month,
        "Interval": event.interval,
        "Days of Week": event.days_of_week
    }
    for event in existing_events
])

existing_events_df.head()

,Name,Start Date,End Date,Recurrent Type,Days of Month,Interval,Days of Week
0,Rent Payment,2024-03-01,None,monthly,[1],1,[]
1,Gym Membership,2024-03-05,None,monthly,[5],1,[]
2,Salary,2024-03-01,None,monthly,"[1, 15]",1,[]
3,Electric Bill,2024-03-10,None,monthly,[10],1,[]
4,Internet Bill,2024-03-15,None,monthly,[15],1,[]


## 4. Export Data

In [7]:
# Export data
print(test_results_df.head(5))
#print(test_tittles_df.head(5))
#test_results_df.to_csv(f"test_cases {datetime.now().strftime("%m%d%Y-%H%M%S")}.csv")
#test_results_df.to_csv(f"test_cases {datetime.now().strftime("%m%d%Y-%H%M%S")}.csv")

  Week Start Date  Event Count  Test Number
0      2024-03-04           15            1
1      2024-03-11           15            1
2      2024-03-18           24            1
3      2024-03-25           23            1
4      2024-04-01           23            1


# How to Test the Results

## Overview
This test verifies that recurring events are generated correctly. It ensures events occur on expected dates, tracks the number of events per week, and compares actual vs. expected results. Three data sources are used:
- **`existing_events_df`** – Current event definitions.
- **`test_cases_data_df`** – Expected parameters for test cases.
- **`test_results_df`** – Actual results from `count_weekly_events`.

---

## 1. Verify Event Occurrences
**Function:** `occurs_on`

### Steps
1. **Check event dates (using `existing_events_df`)**:
   - Ensure each event appears on the correct dates.
   - Example: A weekly event on Mondays should happen every 7 days, always on a Monday.

2. **Validate test case parameters (using `test_cases_data_df`)**:
   - **For n-weekly events:**
     - Ensure they repeat every `n * 7` days.
     - If an event occurs multiple times a week, treat each occurrence separately.

   - **For monthly events:**
     - Check that they occur on the correct day(s) of the month.
     - If a day doesn’t exist in a given month (e.g., 31st in February), confirm behavior:
       - **If `use_last_day` = True**, the event moves to the last valid day.
       - **If `use_last_day` = False**, the event is ignored that month.

✅ **Pass if:** Events occur on expected dates.
❌ **Fail if:** Missing or extra occurrences are found.

---

## 2a. Verify Weekly Event Counts
**Function:** `count_weekly_events`

### Steps
1. **Count weekly events (using `test_results_df`)**:
   - Identify the Monday that starts each week.
   - Count the total number of events in each week.
   - Store the results in a `weekly_count` table with:
     - **Week start date**
     - **Number of events that week**

✅ **Pass if:** Weekly counts match expectations.
❌ **Fail if:** Counts are incorrect (too many or too few events).

---

## 2b. Validate Test Case Results
Once event dates and weekly counts are confirmed, check if the test results match expectations.

### How Test Cases Work
- **Existing Events (`existing_events_df`)**: Contains known valid event parameters.
- **Test Cases (`test_cases_data_df`)**: Defines expected behaviors.
- **Test Results (`test_results_df`)**: Stores actual occurrences after running `count_weekly_events`.

### Steps
#### 1. Check if events repeat correctly (using `test_results_df` and `test_cases_data_df`)
- **For n-weekly events:**
  - Ensure the first occurrence appears in the correct starting week.
  - Confirm events repeat at the correct interval (`n * 7` days).

- **For monthly events:**
  - Verify that they appear on expected days.
  - If a scheduled date doesn’t exist, confirm behavior:
    - **If `use_last_day` = True**, it moves to the last valid day.
    - **If `use_last_day` = False**, it does not appear for that month.
  - Ensure event start weeks align with `test_results_df`.

✅ **Pass if:** Events follow correct recurrence rules.
❌ **Fail if:** Incorrect intervals or missing occurrences.

#### 2. Check the total number of events (using `test_results_df` and `existing_events_df`)
1. Compare event counts for each **Week Start Date** in `test_results_df` against `existing_events_df`.
2. Ensure no weeks have missing or extra occurrences.

✅ **Pass if:** Weekly event counts match expectations.
❌ **Fail if:** There are discrepancies in the event count.

---

## 3. Save Test Logs as CSV Files
Since this test runs in a Jupyter Notebook with Pandas, save structured results.

### Steps
1. **Store results in CSV files:**
   ```python
   import pandas as pd
   from datetime import datetime

   timestamp = datetime.now().strftime("%m%d%Y-%H%M%S")
   test_results_df.to_csv(f"test_cases_{timestamp}.csv", index=False)
   test_cases_data_df.to_csv(f"test_case_parameters_{timestamp}.csv", index=False)
   existing_events_df.to_csv(f"existing_events_{timestamp}.csv", index=False)
   ```

✅ **Pass if:** CSV files contain structured test results.
❌ **Fail if:** Data is missing or incorrectly formatted.

---

## Summary
- **Verify event occurrences** (`occurs_on`) → Use `existing_events_df`.
- **Count weekly event occurrences** (`count_weekly_events`) → Use `test_results_df`.
- **Compare expected vs. actual recurrence** → Use `test_cases_data_df` & `test_results_df`.
- **Ensure correct total event counts** → Compare `test_results_df` and `existing_events_df`.
- **These three DataFrames (`existing_events_df`, `test_cases_data_df`, `test_results_df`) are provided for running the test.**
- **Store results as CSV files** to keep a record of test accuracy.

✅ **Pass if:** Event dates, recurrence, and weekly counts match expected values.
❌ **Fail if:** Discrepancies exist in occurrences or event counts.
